In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
safe_df=pd.read_csv('/content/drive/MyDrive/KDT AI 시즌2/4. 머신러닝 딥러닝/data/breast_cancer_survival.csv')
pd.set_option('display.max_columns',30)
safe_df.head()

,Age,Gender,Protein1,Protein2,Protein3,Protein4,Tumour_Stage,Histology,ER status,PR status,HER2 status,Surgery_type,Date_of_Surgery,Date_of_Last_Visit,Patient_Status
0,42,FEMALE,0.95256,2.15000,0.007972,-0.048340,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,20-May-18,26-Aug-18,Alive
1,54,FEMALE,0.00000,1.38020,-0.498030,-0.507320,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,26-Apr-18,25-Jan-19,Dead
2,63,FEMALE,-0.52303,1.76400,-0.370190,0.010815,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Lumpectomy,24-Aug-18,08-Apr-20,Alive
3,78,FEMALE,-0.87618,0.12943,-0.370380,0.132190,I,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,16-Nov-18,28-Jul-20,Alive
4,42,FEMALE,0.22611,1.74910,-0.543970,-0.390210,II,Infiltrating Ductal Carcinoma,Positive,Positive,Positive,Lumpectomy,12-Dec-18,05-Jan-19,Alive


In [ ]:
safe_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334 entries, 0 to 333
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Age                 334 non-null    int64  
 1   Gender              334 non-null    object 
 2   Protein1            334 non-null    float64
 3   Protein2            334 non-null    float64
 4   Protein3            334 non-null    float64
 5   Protein4            334 non-null    float64
 6   Tumour_Stage        334 non-null    object 
 7   Histology           334 non-null    object 
 8   ER status           334 non-null    object 
 9   PR status           334 non-null    object 
 10  HER2 status         334 non-null    object 
 11  Surgery_type        334 non-null    object 
 12  Date_of_Surgery     334 non-null    object 
 13  Date_of_Last_Visit  317 non-null    object 
 14  Patient_Status      321 non-null    object 
dtypes: float64(4), int64(1), object(10)
memory usage: 39.3+ K

In [ ]:
# safe_df.drop(['Date_of_Surgery','Date_of_Last_Visit','Surgery_type','Histology'],axis=1,inplace=True)

In [ ]:
safe_df['Tumour_Stage'].value_counts()

II     189
III     81
I       64
Name: Tumour_Stage, dtype: int64

In [ ]:
for column in ['Gender', 'Tumour_Stage', 'Histology', 'ER status', 'PR status', 'HER2 status', 'Surgery_type', 'Patient_Status']:
    print(column, safe_df[column].nunique())

Gender 2
Tumour_Stage 3
Histology 3
ER status 1
PR status 1
HER2 status 2
Surgery_type 4
Patient_Status 2


In [ ]:
safe_df = safe_df[safe_df['Gender'] != 'MALE']

In [ ]:
safe_df.drop(['Gender','Date_of_Surgery','Date_of_Last_Visit','PR status','ER status'],axis=1,inplace=True)

In [ ]:
safe_df = safe_df[safe_df['Patient_Status'].notnull()]
safe_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 317 entries, 0 to 333
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             317 non-null    int64  
 1   Protein1        317 non-null    float64
 2   Protein2        317 non-null    float64
 3   Protein3        317 non-null    float64
 4   Protein4        317 non-null    float64
 5   Tumour_Stage    317 non-null    object 
 6   Histology       317 non-null    object 
 7   HER2 status     317 non-null    object 
 8   Surgery_type    317 non-null    object 
 9   Patient_Status  317 non-null    object 
dtypes: float64(4), int64(1), object(5)
memory usage: 27.2+ KB


In [ ]:
# 라벨 인코딩
safe_df['Tumour_Stage'] = safe_df['Tumour_Stage'].map(lambda x: {'I': 1, 'II': 2, 'III': 3}.get(x))
safe_df['Tumour_Stage'].value_counts()

2    179
3     77
1     61
Name: Tumour_Stage, dtype: int64

In [ ]:
safe_df['HER2 status'].value_counts()

Negative    288
Positive     29
Name: HER2 status, dtype: int64

In [ ]:
safe_df['HER2 status'] = safe_df['HER2 status'].map(lambda x: {'Positive': 1, 'Negative': 0}.get(x))
safe_df['HER2 status'].value_counts()

0    288
1     29
Name: HER2 status, dtype: int64

In [ ]:
safe_df['Patient_Status'] = safe_df['Patient_Status'].map({'Alive': 1, 'Dead': 0})

In [ ]:
safe_df['Surgery_type'].value_counts()

Other                          97
Modified Radical Mastectomy    90
Lumpectomy                     66
Simple Mastectomy              64
Name: Surgery_type, dtype: int64

In [ ]:
safe_df['Histology'].value_counts()

Infiltrating Ductal Carcinoma     224
Infiltrating Lobular Carcinoma     81
Mucinous Carcinoma                 12
Name: Histology, dtype: int64

In [ ]:
safe_df=pd.get_dummies(safe_df,columns=['Tumour_Stage','Surgery_type','Histology'])
safe_df.head()

,Age,Protein1,Protein2,Protein3,Protein4,HER2 status,Patient_Status,Tumour_Stage_1,Tumour_Stage_2,Tumour_Stage_3,Surgery_type_Lumpectomy,Surgery_type_Modified Radical Mastectomy,Surgery_type_Other,Surgery_type_Simple Mastectomy,Histology_Infiltrating Ductal Carcinoma,Histology_Infiltrating Lobular Carcinoma,Histology_Mucinous Carcinoma
0,42,0.95256,2.15000,0.007972,-0.048340,0,1,0,1,0,0,0,1,0,1,0,0
1,54,0.00000,1.38020,-0.498030,-0.507320,0,0,0,1,0,0,0,1,0,1,0,0
2,63,-0.52303,1.76400,-0.370190,0.010815,0,1,0,1,0,1,0,0,0,1,0,0
3,78,-0.87618,0.12943,-0.370380,0.132190,0,1,1,0,0,0,0,1,0,1,0,0
4,42,0.22611,1.74910,-0.543970,-0.390210,1,1,0,1,0,1,0,0,0,1,0,0


In [ ]:
safe_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 317 entries, 0 to 333
Data columns (total 17 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Age                                       317 non-null    int64  
 1   Protein1                                  317 non-null    float64
 2   Protein2                                  317 non-null    float64
 3   Protein3                                  317 non-null    float64
 4   Protein4                                  317 non-null    float64
 5   HER2 status                               317 non-null    int64  
 6   Patient_Status                            317 non-null    int64  
 7   Tumour_Stage_1                            317 non-null    uint8  
 8   Tumour_Stage_2                            317 non-null    uint8  
 9   Tumour_Stage_3                            317 non-null    uint8  
 10  Surgery_type_Lumpectomy               

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(safe_df.drop('Patient_Status',axis=1),safe_df['Patient_Status'],test_size=0.2,random_state=2024)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((253, 16), (253,), (64, 16), (64,))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
# Logistic Regression 모델 생성
model = LogisticRegression(random_state=2024)

# 모델 학습
model.fit(X_train, y_train)

# 학습된 모델로 테스트 데이터 예측
y_pred = model.predict(X_test)

# 정확도 계산
accuracy = accuracy_score(y_test, y_pred)

# 결과 출력
print(f"Accuracy: {accuracy}")

Accuracy: 0.78125


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 랜덤 포레스트 모델 생성
rf_model = RandomForestClassifier(random_state=2024)

# 모델 학습
rf_model.fit(X_train, y_train)

# 학습된 모델로 테스트 데이터 예측
y_pred_rf = rf_model.predict(X_test)

# 정확도 계산
accuracy_rf = accuracy_score(y_test, y_pred_rf)

# 결과 출력
print(f"Random Forest Accuracy: {accuracy_rf}")

Random Forest Accuracy: 0.75
